# Chapter5 Data handling(Pandas#2)

## Groupby
- SQL groupby 명령어와 같음
- split -> apply -> combine 과정을 거쳐 연산함

In [1]:
import pandas as pd

In [2]:
# data from: 
ipl_data = {'Team': ['Riders', 'Riders', 'Devils', 'Devils', 'Kings',
         'kings', 'Kings', 'Kings', 'Riders', 'Royals', 'Royals', 'Riders'],
         'Rank': [1, 2, 2, 3, 3,4 ,1 ,1,2 , 4,1,2],
         'Year': [2014,2015,2014,2015,2014,2015,2016,2017,2016,2014,2015,2017],
         'Points':[876,789,863,673,741,812,756,788,694,701,804,690]}

df = pd.DataFrame(ipl_data)
df

,Points,Rank,Team,Year
0,876,1,Riders,2014
1,789,2,Riders,2015
2,863,2,Devils,2014
3,673,3,Devils,2015
4,741,3,Kings,2014
5,812,4,kings,2015
6,756,1,Kings,2016
7,788,1,Kings,2017
8,694,2,Riders,2016
9,701,4,Royals,2014


In [3]:
df.groupby("Team")["Points"].sum()

#결과 : Team을 기준으로 Points을 Sum

Team
Devils    1536
Kings     2285
Riders    3049
Royals    1505
kings      812
Name: Points, dtype: int64

> df.groupby(묶음의 기준이 되는 칼럼)[적용받는 칼럼].적용받는 연산

In [4]:
#한 개이상의 column을 묶을 수 있음

df.groupby(["Team", "Year"])["Points"].sum()

Team    Year
Devils  2014    863
        2015    673
Kings   2014    741
        2016    756
        2017    788
Riders  2014    876
        2015    789
        2016    694
        2017    690
Royals  2014    701
        2015    804
kings   2015    812
Name: Points, dtype: int64

## Hierarchical index
- Groupby 명령의 결과물도 결국은 dataframe
- 두 개의 column으로 groupby를 할 경우, index가 두개 생성

In [5]:
h_index = df.groupby(["Team", "Year"])["Points"].sum()
h_index

Team    Year
Devils  2014    863
        2015    673
Kings   2014    741
        2016    756
        2017    788
Riders  2014    876
        2015    789
        2016    694
        2017    690
Royals  2014    701
        2015    804
kings   2015    812
Name: Points, dtype: int64

In [6]:
h_index.index

MultiIndex(levels=[['Devils', 'Kings', 'Riders', 'Royals', 'kings'], [2014, 2015, 2016, 2017]],
           labels=[[0, 0, 1, 1, 1, 2, 2, 2, 2, 3, 3, 4], [0, 1, 0, 2, 3, 0, 1, 2, 3, 0, 1, 1]],
           names=['Team', 'Year'])

In [7]:
h_index["Devils":"Kings"]

Team    Year
Devils  2014    863
        2015    673
Kings   2014    741
        2016    756
        2017    788
Name: Points, dtype: int64

## Hierarchical index - unstack()
- Group으로 묶여진 데이터를 matrix 형태로 전환해줌

In [8]:
h_index.unstack()

Year,2014,2015,2016,2017
Team,,,,
Devils,863.0,673.0,NaN,NaN
Kings,741.0,NaN,756.0,788.0
Riders,876.0,789.0,694.0,690.0
Royals,701.0,804.0,NaN,NaN
kings,NaN,812.0,NaN,NaN


## Hirarchical index - swaplevel
- Index level을 변경할 수 있음

In [9]:
h_index.swaplevel()

Year  Team  
2014  Devils    863
2015  Devils    673
2014  Kings     741
2016  Kings     756
2017  Kings     788
2014  Riders    876
2015  Riders    789
2016  Riders    694
2017  Riders    690
2014  Royals    701
2015  Royals    804
      kings     812
Name: Points, dtype: int64

In [10]:
h_index.swaplevel().sortlevel(0)

Year  Team  
2014  Devils    863
      Kings     741
      Riders    876
      Royals    701
2015  Devils    673
      Riders    789
      Royals    804
      kings     812
2016  Kings     756
      Riders    694
2017  Kings     788
      Riders    690
Name: Points, dtype: int64

## Hierarchical index - operations
- index level을 기준으로 기본 연산 수행 가능

In [11]:
h_index.sum(level=0)

Team
Devils    1536
Kings     2285
Riders    3049
Royals    1505
kings      812
Name: Points, dtype: int64

In [12]:
h_index.sum(level=1)

Year
2014    3181
2015    3078
2016    1450
2017    1478
Name: Points, dtype: int64

## Groupby - grouped
- Groupby에 의해 Split된 상태를 추출 가능함

In [13]:
grouped = df.groupby("Team")

for name, group in grouped:
    print(name)
    print(group)
    
#Tuple 형태로 그룹의 key값, value값이 추출됨

Devils
   Points  Rank    Team  Year
2     863     2  Devils  2014
3     673     3  Devils  2015
Kings
   Points  Rank   Team  Year
4     741     3  Kings  2014
6     756     1  Kings  2016
7     788     1  Kings  2017
Riders
    Points  Rank    Team  Year
0      876     1  Riders  2014
1      789     2  Riders  2015
8      694     2  Riders  2016
11     690     2  Riders  2017
Royals
    Points  Rank    Team  Year
9      701     4  Royals  2014
10     804     1  Royals  2015
kings
   Points  Rank   Team  Year
5     812     4  kings  2015


In [14]:
#특정 key값을 가진 그룹의 정보만 추출 가능

grouped.get_group("Devils")

,Points,Rank,Team,Year
2,863,2,Devils,2014
3,673,3,Devils,2015


- 추출된 group 정보에는 세 가지 유형의 apply가 가능함
- Aggregation: 요약된 통계정보를 추출해 줌
- Transformation: 해당 정보를 변환해줌
- Filtration: 특정 정보를 제거하여 보여주는 필터링 기능

### aggregation

In [15]:
grouped.agg(sum)

,Points,Rank,Year
Team,,,
Devils,1536,5,4029
Kings,2285,5,6047
Riders,3049,7,8062
Royals,1505,5,4029
kings,812,4,2015


In [17]:
import numpy as np
grouped.agg(np.mean)

,Points,Rank,Year
Team,,,
Devils,768.000000,2.500000,2014.500000
Kings,761.666667,1.666667,2015.666667
Riders,762.250000,1.750000,2015.500000
Royals,752.500000,2.500000,2014.500000
kings,812.000000,4.000000,2015.000000


In [18]:
#특정 칼럼에 여라개의 function을 Apply할 수도 있음
grouped["Points"].agg([np.sum, np.mean, np.std])

,sum,mean,std
Team,,,
Devils,1536,768.000000,134.350288
Kings,2285,761.666667,24.006943
Riders,3049,762.250000,88.567771
Royals,1505,752.500000,72.831998
kings,812,812.000000,NaN


### transformation
- Aggregation과 달리 key값 별로 요약된 정보가 아님
- 개별 데이터의 변환을 지원함

In [28]:
df

,Points,Rank,Team,Year
0,876,1,Riders,2014
1,789,2,Riders,2015
2,863,2,Devils,2014
3,673,3,Devils,2015
4,741,3,Kings,2014
5,812,4,kings,2015
6,756,1,Kings,2016
7,788,1,Kings,2017
8,694,2,Riders,2016
9,701,4,Royals,2014


In [26]:
score = lambda x: (x)
grouped.transform(score)

,Points,Rank,Year
0,876,1,2014
1,789,2,2015
2,863,2,2014
3,673,3,2015
4,741,3,2014
5,812,4,2015
6,756,1,2016
7,788,1,2017
8,694,2,2016
9,701,4,2014


In [38]:
#단 max나 min처럼 Series 데이터에 적용되는 데이터들은 Key값을 기준으로 Grouped된 데이터 기준
_max = lambda x: (x.max())
df.groupby('Team').transform(_max)

,Points,Rank,Year
0,876,2,2017
1,876,2,2017
2,863,3,2015
3,863,3,2015
4,788,3,2017
5,812,4,2015
6,788,3,2017
7,788,3,2017
8,876,2,2017
9,804,4,2015


In [33]:
score = lambda x: (x - x.mean()) / x.std()
grouped.transform(score)

,Points,Rank,Year
0,1.284327,-1.500000,-1.161895
1,0.302029,0.500000,-0.387298
2,0.707107,-0.707107,-0.707107
3,-0.707107,0.707107,0.707107
4,-0.860862,1.154701,-1.091089
5,NaN,NaN,NaN
6,-0.236043,-0.577350,0.218218
7,1.096905,-0.577350,0.872872
8,-0.770596,0.500000,0.387298
9,-0.707107,0.707107,-0.707107


### filter
- 특정 조건으로 데이터를 검색할 때 사용
- filter 안에는 boolean 조건이 존재해야함
- len(x)는 grouped된 dataframe 개수

In [34]:
df.groupby('Team').filter(lambda x: len(x) >= 3)

,Points,Rank,Team,Year
0,876,1,Riders,2014
1,789,2,Riders,2015
4,741,3,Kings,2014
6,756,1,Kings,2016
7,788,1,Kings,2017
8,694,2,Riders,2016
11,690,2,Riders,2017


In [39]:
df.groupby('Team').filter(lambda x: x["Rank"].sum() > 2)

,Points,Rank,Team,Year
0,876,1,Riders,2014
1,789,2,Riders,2015
2,863,2,Devils,2014
3,673,3,Devils,2015
4,741,3,Kings,2014
5,812,4,kings,2015
6,756,1,Kings,2016
7,788,1,Kings,2017
8,694,2,Riders,2016
9,701,4,Royals,2014


## Pivot table Crosstab

### Pivot Table

### Crosstab
- 특히 두 칼럽에 교차 빈도, 비율, 덧셈 등을 구할 때 사용
- Pivot table의 특수한 형태
- User-Item Rating Matrix 등을 만들 때 사용가능함

## Merge & Concat

### Merge
- SQL에서 많이 사용하는 Merge와 같은 기능
- 두 개의 데이터를 하나로 합침